# BBP-WORKFLOW: Simulation Campaign Analysis

Overview:
* Manual access to bbp-workflow simulation campaign based on Nexus URL
* Quick analysis/visualizations
* Test & development of analysis routines (before putting them into workflow tasks)

Requirements:
* venv with bluepy and bbp-workflow, e.g. to be used in JupyterLab on BB5:
~~~
/gpfs/bbp.cscs.ch/apps/tools/jupyter/create-jupyterlab-venv ~/BbpWorkflowKernel archive/xxxx-xx py-bluepy py-bbp-workflow
~~~

In [5]:
from entity_management.simulation import SimulationCampaignConfiguration
# from bbp_workflow.utils import xr_from_dict # [DEPRECATED]
from bbp_workflow.simulation.util import xr_from_dict
from bluepy import Circuit
from bluepy import Cell
from bluepy import Simulation
import os.path
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd
import pickle
from scipy.optimize import curve_fit, OptimizeWarning
import sys
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=OptimizeWarning)

## Load simulation campaign config from Nexus Knowledge Graph

In [2]:
# Nexus instance and simulation campaign URL
# kg_base = 'https://staging.nise.bbp.epfl.ch/nexus/v1'
# kg_org = 'bbp_test'
# kg_proj = 'christoph'
# url = 'https://staging.nise.bbp.epfl.ch/nexus/v1/resources/bbp_test/christoph/_/544b3ea7-e1c5-48c9-8468-2bf61503e272'
# url = 'https://staging.nexus.ocp.bbp.epfl.ch/v1/resources/bbp_test/christoph/_/3b9c2546-af67-442b-bee0-4930b4828a62'
# url = 'https://staging.nexus.ocp.bbp.epfl.ch/v1/resources/bbp_test/christoph/_/231822cd-0375-4166-a990-2e2ee3cc27c8'
# url = 'https://staging.nexus.ocp.bbp.epfl.ch/v1/resources/bbp_test/christoph/_/5b1420ca-dd31-4def-96d6-46fe99d20dcc' # [Andras's plasticity]
# url = 'https://staging.nexus.ocp.bbp.epfl.ch/v1/resources/bbp_test/christoph/_/4073e95f-abb1-4b86-8c38-13cf9f00ce0b' # [Andras's plasticity]
# url = 'https://staging.nexus.ocp.bbp.epfl.ch/v1/resources/bbp_test/christoph/_/051f2d42-1405-4ef5-8e1d-9118766494b9' # [Andras's plasticity]
# url = 'https://staging.nexus.ocp.bbp.epfl.ch/v1/resources/bbp_test/christoph/_/d48e2d22-5a48-4fb7-92da-d6def6591f95'
# url = 'https://staging.nexus.ocp.bbp.epfl.ch/v1/resources/bbp_test/christoph/_/94dc6820-6760-43df-85b7-e3b7e823ce9a'
# url = 'https://staging.nexus.ocp.bbp.epfl.ch/v1/resources/bbp_test/christoph/_/5b4b6bb3-2379-45b2-9c44-02059c1d4aa2'
# url = 'https://staging.nexus.ocp.bbp.epfl.ch/v1/resources/bbp_test/christoph/_/fba7275b-04ce-4932-908b-6f37a4d1705b'
# url = 'https://staging.nexus.ocp.bbp.epfl.ch/v1/resources/bbp_test/christoph/_/efe40a99-e828-4808-94ee-a56eca3cb1e1'
# url = 'https://staging.nexus.ocp.bbp.epfl.ch/v1/resources/bbp_test/christoph/_/427548ec-22ab-4e80-98a1-9e261ab7d51a'
# url = 'https://staging.nexus.ocp.bbp.epfl.ch/v1/resources/bbp_test/christoph/_/12bf31ac-c650-40e8-b470-79c36b9552b7'
kg_base = None
kg_org = None
kg_proj = 'somatosensorycortex'


# Visual contrast
url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/3244ae1f-e48a-49f5-86c0-f9c92fb4b076' # Baseline scan incl. tuning with James' Ca 1.05 conductance settings
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/9dd5a488-6020-4f3f-b32e-031c46a903a4' # OptoTest w/o opto
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/7fca28e8-be50-4cc8-8cc0-d309c14f0786' # OptoTest with 0% depol
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/9e9ef9bb-568d-4ea5-826d-757e1da80edb' # OptoTest with 0% depol and PercentEnd added
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/56a11f07-aba3-45ca-a73d-19202a29347b' # OptoTest with 0% depol (full tuning)
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/218c9553-3154-4b4f-a2ee-ccf5f5308e30' # Baseline scan test with full tuning

# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/4086d78b-e2b4-41b5-b40a-a8f130b982b3'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/5e6b29a6-4cf8-44b0-a6e4-30f35a1a01ed'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/e873a484-b1e7-4d72-b6a8-9ce873004d36'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/c02cc5f8-8b78-40ec-9cde-04e765475357'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/f78b83a5-ce42-4bb2-b25c-71a753deaf78'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/aed17b53-42ea-4f99-abfb-d667a5d55598'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/db4bfc86-b40b-46a3-983a-e7e084f227ff'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/bbc32904-495d-457d-a297-7b34bd8d7bd3'

# Simplified connectome models
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/b1299740-afe5-4d50-ad2a-f54dbb80c08a' # Toposample base [OLD]
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/78140838-af74-4a90-b30a-b6aeb388f575' # Toposample base [OLD2]
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/1892cab7-67ea-457b-a3d3-710f3c1f6fff' # Toposample base [OLD3]
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/9e111b3c-876c-4285-b99d-0fa44626ba67' # Toposample base scan

# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/43269208-a4e3-4b21-9d25-fa97c23399d9' # Toposample 5th order
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/c5ec762f-4146-43ae-be66-873ab1434087' # Toposample 4th order
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/2afaaa6c-8c86-4aea-82b7-d493930fa451' # Toposample 3rd order
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/abc8c760-839b-40d2-aaf8-efd8e9b2639b' # Toposample 2nd order
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/8165329d-f63c-4b50-adb7-f282f2b3f3b4' # Toposample 1st order
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/2bc672d4-1f3a-413d-bcff-1a4f6ccd7516' # Toposample base

# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/831dfef0-ce93-48f1-9f62-0f632f5c13ef' # Evoked 5th order
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/c4fcea8c-1673-45f4-ab49-20cbe5627eee' # Evoked 4th order
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/5cd39608-3eea-44b8-aa77-d4e750ecfab1' # Evoked 3rd order
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/6bce01b8-2425-4559-8721-de927478be30' # Evoked 2nd order
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/21e8e6ae-a8b0-4908-8fa6-a8e448051c5a' # Evoked 1st order
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/0d890eb1-ff37-400b-b22b-20974f3aad50' # Evoked base

# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/27f76bd4-3aee-42f2-b0a3-fdcf5bb52d38' # Spont 5th order
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/29cb305e-1044-476d-8c5f-5c926e2c1d9c' # Spont 4th order
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/4e2f6fbb-3b4a-4714-bcd6-7e8d1aa4b7d3' # Spont 3rd order
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/686ade18-72e8-45a4-a379-25c57ddaf4c6' # Spont 2nd order
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/c441a606-906b-4290-863e-309a9e97af6b' # Spont 1st order
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/a29b3787-cbfb-48aa-8e9d-ae43f9278d32' # Spont base

# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/6e9b4c94-2a43-4b34-9e76-0a209f89f381'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/890f57f3-f07b-47a7-be73-014367de4df7'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/b87aac8d-b3d0-4e63-b589-560d826bbfdc'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/ef21a1bb-8138-4438-b945-9d5b11fe7b49'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/0c81551e-037b-4d69-92f3-00f33c16b5f1'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/df29e24b-73ba-4210-975b-8c7389f1ddc7'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/332fbe47-22f7-48a7-a7fb-5ff13fa5f6e5'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/eb22c639-34d7-4aa3-8113-3074025dd9f5'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/e0edb4df-5c8c-412c-ad3f-2d4cdbbd454c'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/39637809-9b5f-4753-bf2f-e0edc2fe4bc0'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/4f2f7e41-f6a4-4d86-9e3d-03484bd34761'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/6f0a9a9b-842b-4874-8b16-6c6010c07ace'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/4068aac5-4e36-4fb3-8ee8-5d612f13c2dd'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/1978b10b-1508-4584-a71a-2fb8a4809d7a'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/8336f8f4-8256-4207-bbe7-d61c9455b5fa'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/ab3f0887-44bf-4f37-a169-c7b7e6a967e7'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/08ecfd3a-7ebb-4f72-8d62-b8e83211b9b4'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/73becc80-9d25-4d0e-b82c-eca7526b9cf6'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/b0a99807-f1c4-41e0-a261-5fb4e718412d'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/fa4830f8-44e5-4d7f-a41f-6f6ae828030d'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/ce6a66c4-87bd-4008-87bc-009939b45da2'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/413cb2ae-92b1-4070-be37-6d105373cc8c'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/85a5dba0-e352-46c4-a424-485603155be7'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/2b20c216-9a68-4f0c-b6ec-4eb917df34cd'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/f553e919-2fac-460c-a58e-3facc388da90'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/2b193707-c91c-48b6-aa51-06f02e23c2ab'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/2846f4c3-de17-41b2-8983-0e9d70bf6699'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/0d36641c-2b97-4bcc-a7e1-170498dc40bc'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/407ce5ba-fbc1-4b04-b63c-b48d85507102'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/931e95f8-00ec-4e88-bf0c-ba87e706e5aa'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/bc0a795f-9bf2-48bb-943a-c43a332bffe8'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/1bb04873-ad1b-4cf0-abf7-5bca6159d956'
# url = 'https://bbp.epfl.ch/nexus/v1/resources/bbp/somatosensorycortex/_/52243b3c-395c-4c7e-95fa-eec65edaee33'

# Nexus token, to be retrieved from https://bbp.epfl.ch/nexus/web/ => 'Copy token'
token = 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICI5T0R3Z1JSTFVsTTJHbFphVDZjVklnenJsb0lzUWJmbTBDck1icXNjNHQ4In0.eyJleHAiOjE2NjQyODk4OTAsImlhdCI6MTY2NDI2NTE0NiwiYXV0aF90aW1lIjoxNjY0MjYxMDkwLCJqdGkiOiIzZTgyYTY0Mi03N2NhLTQ2ZDctOGY5Zi04NGY4ZGZhMDNiZTQiLCJpc3MiOiJodHRwczovL2JicGF1dGguZXBmbC5jaC9hdXRoL3JlYWxtcy9CQlAiLCJhdWQiOiJodHRwczovL3NsYWNrLmNvbSIsInN1YiI6ImY6MGZkYWRlZjctYjJiOS00OTJiLWFmNDYtYzY1NDkyZDQ1OWMyOnBva29ybnkiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJiYnAtbmlzZS1uZXh1cy1mdXNpb24iLCJub25jZSI6ImY4YjQ3NDdlNWYyYzQ4ZDNhZTZjZDNhZGVmZDVjNjE0Iiwic2Vzc2lvbl9zdGF0ZSI6ImNhMmViY2E5LWQ1NGItNDA1Yy1iMGNmLWNiNmMzOTkzMzdjYyIsImFjciI6IjAiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1c2VyIiwiZGVmYXVsdC1yb2xlcy1iYnAiXX0sInJlc291cmNlX2FjY2VzcyI6eyJodHRwczovL3NsYWNrLmNvbSI6eyJyb2xlcyI6WyJzbGFja191c2VyX3JvbGUiXX19LCJzY29wZSI6Im9wZW5pZCBuZXh1cyBwcm9maWxlIGxvY2F0aW9uIGVtYWlsIiwic2lkIjoiY2EyZWJjYTktZDU0Yi00MDVjLWIwY2YtY2I2YzM5OTMzN2NjIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsIm5hbWUiOiJDaHJpc3RvcGggUG9rb3JueSIsImdyb3VwcyI6WyIvYmJwLWRldi1wcm9qMTA5IiwiL2JicC1zdmMtb3BlbnNoaWZ0IiwiL2JicC11c2VyLXByb2o5NiIsIi9iYnAtdXNlci1wcm9qOTk5OSIsIi9iYnAtZGV2LXByb2oxNDgiLCIvYmJwLXVzZXItcHJvajY0IiwiL2JicC11c2VyLXJlbGVhc2UtbDIiLCIvYmJwLXVzZXItcHJvajU5IiwiL2JicC1kZXYtcHJvajkiLCIvYmJwLXVzZXItY29yZWJsdXJvbiIsIi9iYnAtdXNlci1wcm9qMzIiLCIvYmJwLWRldi1wcm9qMTQxIiwiL2JicC1vdS1zaW11bGF0aW9ubmV1cm9zY2llbmNlIiwiL2JicC1kcy1leHBlcmltZW50IiwiL2JicC1zdmMtbHh2aXoiLCIvYmJwLXVzZXItcHJvajEwNiIsIi9iYnAtc3ZjLXZpeiIsIi9iYnAtdXNlci1yZWxlYXNlLWwxIiwiL2JicC1kZXYtcHJvajEzNCIsIi9iYnAtZnVsbCIsIi9iYnAtc3ZjLWdpdGxhYiIsIi9iYnAtY29sbGFiLWxpbmthYmxlLWFjY291bnRzIiwiL2JicC11c2VyLXByb2oxMTIiLCIvYmJwLWRldi1yZWZpbmVtZW50cHJvcCIsIi9iYnAtZGV2LXByb2o4MyIsIi9iYnAtb3UtY29ubmVjdG9taWNzIiwiL2JicC1zdmMtYmc0IiwiL2JicC1kZXYtcHJvajgyIiwiL2JicC1zcnYtbGluc3J2MiIsIi9iYnAtdXNlci1wcm9qOTQiLCIvYmJwLXVzZXItcHJvajEyMyIsIi9iYnAtZHMtaHBjIiwiL2JicC11c2VyLXJlbGVhc2UtbDAiLCIvYmJwLXVzZXItcHJvajY4IiwiL2JicF9jb25mbHVlbmNlIiwiL2JicC1vdS1lcGZsIiwiL2JicC11c2VyLXByb2oxMTQiLCIvYmJwLXN2Yy1ocGMiLCIvYmJwLXN2Yy1jb2RlIiwiL2JicC1kZXYtcHJvajEzMiIsIi9iYnAtc3ZjLXNsYWNrIiwiL2JicC1kZXYtcHJvajEwMiIsIi9iYnAtc3RhZmYiLCIvYmJwLWRldi1oYnB2aXoiLCIvYmJwLWRldi1wcm9qZWN0cHJvcCIsIi9iYnAtdXNlci1wcm9qODQiLCIvYmJwLXN2Yy1rdWJlcm5ldGVzIiwiL2JicC1zdmMtc3RhdHVzIiwiL2JicC11c2VyLXByb2oxIiwiL2JicC1wdWJsaWNhdGlvbnMiLCIvYmJwLXVzZXItcHJvajMiXSwibG9jYXRpb24iOiJCMSA1IDI3OS4wNTAiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJwb2tvcm55IiwiZ2l2ZW5fbmFtZSI6IkNocmlzdG9waCIsImZhbWlseV9uYW1lIjoiUG9rb3JueSIsImVtYWlsIjoiY2hyaXN0b3BoLnBva29ybnlAZXBmbC5jaCJ9.MyTwAUqj_3YdRcb2I7aWME38mWyyv5KQ6jPzXdAHQXhX5wisfxZMDpNQEpA0Ailhm5_9LJOjAIqf8c09O0A-ojEbTyhPYAe-l3kW6tZJ8urQO51A6qKd2GUzKx-5xKKU78IXQpaF59Kl-yvpp1OyhYPUmSzbgN6l2GWJqfdpATdnltvKWh9MHE_J49sSjxgrYJ-FZU3wJQ8bwPyfPblmto3wwoSy2X3omhLaEDAbNJE-YET8JWTEy-YTNwH5_KEX8smzCmqNRAeP6P-PezFI_uRkzClq4YGF-bFE7lQWGv9BLeUx-MA1Gd9BZXXqbSkfX6lrDOR-a5k8UieF9GwjNw'

# Settings
stim_name = 'Stimulus_spikeReplay' # Name of stimulus to be plotted (optional)
bin_size = 50 # Time resolution (ms) of firing rate histograms
psth_bin_size = 5 # Time resolution (ms) of stimulus PSTHs
psth_interval = [-50, 1000] # Time interval (ms) rel. to stim. onset
response_interval = [0, 1000] # Time interval (ms) rel. to stim. onset
save_figs = True

# Load campaign and show overview
sim_campaign_cfg = SimulationCampaignConfiguration.from_url(url=url, base=kg_base, org=kg_org, proj=kg_proj, use_auth=token) # retrieve KG entity
config_dict = sim_campaign_cfg.configuration.as_dict(use_auth=token) # get sim campaign config as dict
config = xr_from_dict(config_dict)  # get sim campaign config as Xarray
save_path_base = config.attrs['path_prefix']
print(f'Simulation campaign configuration "{sim_campaign_cfg.name}" with {config.to_series().count()} simulations loaded!')

idx_names = config.to_series().index.names
if len(idx_names) == 1:
    idx_counts = config.to_series().index.shape
else:
    idx_counts = config.to_series().index.levshape
coords_str = ', '.join([f'{n} ({c})' for n, c in zip(idx_names, idx_counts)])
print(f'Coordinate(s) found: {coords_str}')

Simulation campaign configuration "SSCx-Bio_M-20200805-hex0-DriftGratStim_Baseline_CondCa1p05FrProp0p5DepolRatio0p5" with 63 simulations loaded!
Coordinate(s) found: sparsity (3), rate_bk (3), rate_max (7), seed (1)


In [57]:
# # OPTIONAL: Change save path (in case base folder is read-only)
# save_path_base = '/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/analyses/proj96'

## Analysis part of code
[which could be integrated as a bbp-workflow task]

In [ ]:
# Spike train visualization
c = Circuit(config.circuit_config)
layers = np.unique(c.cells.get(properties=Cell.LAYER))
lay_colors = np.vstack((plt.cm.Greens(0.5), plt.cm.Purples(0.5)))
lay_colors = np.vstack([lay_colors[np.mod(i, lay_colors.shape[0])] for i in range(len(layers))])
lay_ids = [c.cells.ids({Cell.LAYER: lay}) for lay in layers]
gid_min = min(c.cells.ids())
gid_max = max(c.cells.ids())
rate_lw = 0.5 # Linewidth

plot_ids = np.full(gid_max - gid_min + 1, -1).astype(int)
gid_offset = gid_min
lay_offset = 0
for lidx in range(len(layers)):
    plot_ids[lay_ids[lidx] - gid_offset] = np.arange(len(lay_ids[lidx])) + lay_offset
    lay_offset = lay_offset + len(lay_ids[lidx])

for idx, (data_idx, data_path) in enumerate(config.to_series().iteritems()):
    if not isinstance(data_idx, tuple):
        data_idx = (data_idx,)  # if index is atomic (just a number) make tuple out of it

    print('Processing {}...'.format(data_path))
    try:
        sim_path = os.path.join(config.path_prefix, data_path)
        blue_config_file = os.path.join(sim_path, 'BlueConfig')
        sim = Simulation(blue_config_file)

        plt.figure(figsize=(18, 6), dpi=300)
        for lidx, lay in enumerate(layers):
            lay_spikes = sim.spikes.get(gids=lay_ids[lidx])
#             plt.plot(lay_spikes.index, plot_ids[lay_spikes.values - gid_offset], '.', markersize=3.0, markeredgecolor='none', color=lay_colors[lidx, :])
            plt.plot(lay_spikes.index, plot_ids[lay_spikes.values - gid_offset], ',', markersize=1.0, markeredgecolor='none', color=lay_colors[lidx, :])
            plt.text(0, np.mean(plot_ids[lay_ids[lidx] - gid_offset]), f'L{lay}', color=lay_colors[lidx, :], ha='right', va='center')

        label = '  '.join(f'{i[0]}={i[1]}' for i in zip(idx_names, data_idx))
        plt.title(sim_campaign_cfg.name + '\n' + label, fontweight='bold')
        plt.yticks([])
        plt.xlim((0, config.sim_duration)) # duration/sim_duration
        plt.ylim((gid_min, gid_max))
        plt.gca().invert_yaxis()
        plt.xlabel('Time (ms)')
        plt.ylabel('GIDs\n')
        plt.text(min(plt.xlim()), min(plt.ylim()), f'{sim.spikes.get().size} spikes', ha='left', va='bottom', fontsize='x-small')

        ax_sim = plt.gca()
        ax_rate = plt.gca().twinx()
        lines = []
        h = sim.plot.firing_rate_histogram(group=np.intersect1d(sim.target_gids, c.cells.ids('Excitatory')), sample=None, binsize=bin_size, label='EXC', ax=ax_rate)
        lines += list(filter(lambda obj: obj.get_label() == 'EXC', h.get_children()))
        h = sim.plot.firing_rate_histogram(group=np.intersect1d(sim.target_gids, c.cells.ids('Inhibitory')), sample=None, binsize=bin_size, label='INH', ax=ax_rate)
        lines += list(filter(lambda obj: obj.get_label() == 'INH', h.get_children()))
        for l in lines:
            l.set_linewidth(rate_lw)
        lgd_ncol = 2
        ax_rate.set_ylabel('Firing rates (Hz)')

        if stim_name is not None and stim_name in sim.config:
            stim_spike_file = os.path.normpath(os.path.join(sim_path, sim.config[stim_name]['SpikeFile']))
            stim_cfg_file = os.path.splitext(stim_spike_file)[0] + '.json'
            if os.path.exists(stim_spike_file):
                if os.path.exists(stim_cfg_file):
                    with open(stim_cfg_file, 'r') as f:
                        stim_cfg = json.load(f)
                    if 'props' in stim_cfg.keys() and 'time_windows' in stim_cfg['props'] and 'stim_train' in stim_cfg['props']:
                        # Plot stimulus time windows
                        time_windows = stim_cfg['props']['time_windows']
                        stim_train = stim_cfg['props']['stim_train']

                        num_patterns = max(stim_train) + 1
                        num_stimuli = len(stim_train)

                        y_lim = ax_sim.get_ylim()
                        pat_colors = plt.cm.jet(np.linspace(0, 1, num_patterns))
                        for stim_idx, pat_idx in enumerate(stim_train):
                            ax_sim.plot(np.full(2, time_windows[stim_idx]), [y_lim[0] - 0.02 * np.diff(y_lim)[0], y_lim[1] + 0.02 * np.diff(y_lim)[0]], color=pat_colors[pat_idx, :], zorder=0, clip_on=False)

                # Plot input firing rates from spike file
                num_bins = np.round(sim.t_end / bin_size).astype(int)
                bins = np.arange(num_bins + 1) * bin_size

                stim_spikes = pd.read_csv(stim_spike_file, sep='\t')
#                 stim_spikes = pd.read_csv(stim_spike_file, sep=' ')
                num_fibers = len(np.unique(stim_spikes.values))

                stim_rate, _ = np.histogram(stim_spikes.index, bins=bins)
                stim_rate = (1e3 * stim_rate / bin_size) / num_fibers # Rate (Hz)

                ax_rate.step(bins, np.hstack((stim_rate[0], stim_rate)), where='pre', color='k', lw=rate_lw, label='STIM', zorder=0)
                lgd_ncol = lgd_ncol + 1

        plt.legend(loc='lower right', ncol=lgd_ncol, fontsize='xx-small', frameon=False, borderpad=0, borderaxespad=0, columnspacing=1, bbox_to_anchor=(1.0, 1.0))
        plt.tight_layout()
        if save_figs:
            save_path = os.path.join(save_path_base, config.name, 'analyses')
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            output_file = os.path.join(save_path, sim_campaign_cfg.name + f'__SpikeOverview_{idx}.png')
            plt.savefig(output_file)
    except FileNotFoundError:
        print('Results file not found ... SKIPPING')
    except:
        print("Unexpected error:", sys.exc_info()[0])


In [59]:
# # Manually define stimuli (single pattern with certain number of repetitions) [OPTIONAL]
# num_stim = 10
# ISI = 1000 # (ms)
# T0 = 2000 # (ms)
# stim_train = np.zeros(num_stim, dtype=int)
# time_windows = np.arange(T0, T0 + (num_stim + 1) * ISI, ISI)
# num_patterns = np.max(stim_train) + 1

In [ ]:
# Active fraction of population
from scipy.signal import find_peaks

osc_color = 'tab:red'
fract_th = None # 0.25 # Threshold for oscillation peak detection (fraction of active cells)
bin_size = 50 # (ms)
num_bins = np.round(config.sim_duration / bin_size).astype(int)
bins = np.arange(num_bins + 1) * bin_size

for idx, (data_idx, data_path) in enumerate(config.to_series().iteritems()):
    if not isinstance(data_idx, tuple):
        data_idx = (data_idx,)  # if index is atomic (just a number) make tuple out of it

    print('Processing {}...'.format(data_path))
    sim_path = os.path.join(config.path_prefix, data_path)
    blue_config_file = os.path.join(sim_path, 'BlueConfig')    
    sim = Simulation(blue_config_file)
    spk = sim.spikes.get()

    num_spiking_cells = []
    for bidx in range(num_bins):
        spk_sel = spk[np.logical_and(spk.index >= bins[bidx], spk.index < bins[bidx + 1])]
        num_spiking_cells.append(len(np.unique(spk_sel.values)))

    num_spiking_cells = np.array(num_spiking_cells)
    total_num_cells = len(sim.target_gids)
    fraction_active = num_spiking_cells / total_num_cells

    # Simple oscillation detection (peak detection above theshold)
    if fract_th is not None:
        peak_ids = find_peaks(fraction_active, height=fract_th)[0]
        peak_times = 0.5 * (bins[peak_ids] + bins[peak_ids + 1])

        # Burst statistics
        osc_count = len(peak_times)
        osc_intervals = np.diff(peak_times) # Time between oscialltion peaks (ms)
        osc_T = np.mean(osc_intervals) # Mean oscillation period (ms)
        osc_f = 1e3 / osc_T # Mean oscillation frequency (Hz)
        osc_CV = np.std(osc_intervals) / osc_T # Coefficient of variation

    # Plot active fraction
    plt.figure(figsize=(18, 6), dpi=300)
    plt.step(bins, 100.0 * fraction_active[[0] + list(range(num_bins))])
    plt.xlim(plt.xlim()) # Freeze axis limits
    if fract_th is not None:
        plt.plot(0.5 * (bins[peak_ids] + bins[peak_ids + 1]), 100.0 * fraction_active[peak_ids], 'x', color=osc_color, label='Peaks')
        plt.plot(plt.xlim(), np.full(2, 100.0 * fract_th), '--', color=osc_color, zorder=0, label='Threshold')

    label = '  '.join(f'{i[0]}={i[1]}' for i in zip(idx_names, data_idx))
    plt.title(sim_campaign_cfg.name + '\n' + label, fontweight='bold')
    plt.xlim((0, config.sim_duration)) # duration
    plt.ylim((0, 100))
    plt.ylabel('Active fraction (%)')
    plt.xlabel('Time (ms)')
    if fract_th is not None:
        plt.text(np.max(plt.xlim()), 0.99 * np.max(plt.ylim()), f'N={osc_count} events, T={osc_T:.1f}ms, f={osc_f:.1f}Hz, CV={osc_CV:.2f} ', color=osc_color, fontweight='bold', ha='right', va='top')

    if stim_name is not None and stim_name in sim.config:
        stim_spike_file = os.path.normpath(os.path.join(sim_path, sim.config[stim_name]['SpikeFile']))
        stim_cfg_file = os.path.splitext(stim_spike_file)[0] + '.json'
        if os.path.exists(stim_spike_file) and os.path.exists(stim_cfg_file):
            with open(stim_cfg_file, 'r') as f:
                stim_cfg = json.load(f)
            if 'props' in stim_cfg.keys() and 'time_windows' in stim_cfg['props'] and 'stim_train' in stim_cfg['props']:
                # Plot stimulus time windows
                time_windows = stim_cfg['props']['time_windows']
                stim_train = stim_cfg['props']['stim_train']

                num_patterns = max(stim_train) + 1
                num_stimuli = len(stim_train)

                y_lim = plt.gca().get_ylim()
                pat_colors = plt.cm.jet(np.linspace(0, 1, num_patterns))
                for stim_idx, pat_idx in enumerate(stim_train):
                    plt.plot(np.full(2, time_windows[stim_idx]), [y_lim[0] - 0.02 * np.diff(y_lim)[0], y_lim[1] + 0.02 * np.diff(y_lim)[0]], color=pat_colors[pat_idx, :], zorder=0, clip_on=False)
#         ###
#         # Manually defined stimuli
#         y_lim = plt.gca().get_ylim()
#         pat_colors = plt.cm.jet(np.linspace(0, 1, num_patterns))
#         for stim_idx, pat_idx in enumerate(stim_train):
#             plt.plot(np.full(2, time_windows[stim_idx]), [y_lim[0] - 0.02 * np.diff(y_lim)[0], y_lim[1] + 0.02 * np.diff(y_lim)[0]], color=pat_colors[pat_idx, :], zorder=0, clip_on=False)
#         ###

    plt.tight_layout()
    if save_figs:
        save_path = os.path.join(save_path_base, config.name, 'analyses')
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        output_file = os.path.join(save_path, sim_campaign_cfg.name + f'__PopulationEvents_{idx}.png')
        plt.savefig(output_file)


In [ ]:
# Population response PSTHs (per layer and Exc/INH)

c = Circuit(config.circuit_config)
layers = np.unique(c.cells.get(properties=Cell.LAYER))
layer_colors = plt.cm.jet(np.linspace(0, 1, len(layers)))
syn_classes = np.unique(c.cells.get(properties=Cell.SYNAPSE_CLASS))
syn_ls = ['-', '--'] # Linestyles

t_len = np.diff(psth_interval)[0]
num_bins = np.round(t_len / psth_bin_size).astype(int)
bins = np.arange(num_bins + 1) * psth_bin_size + psth_interval[0]

for idx, (data_idx, data_path) in enumerate(config.to_series().iteritems()):
    if not isinstance(data_idx, tuple):
        data_idx = (data_idx,)  # if index is atomic (just a number) make tuple out of it
    try:
        
        #Load stim config file
        sim_path = os.path.join(config.path_prefix, data_path)
        blue_config_file = os.path.join(sim_path, 'BlueConfig')
        
        sim = Simulation(blue_config_file)        
        stim_spike_file = os.path.normpath(os.path.join(sim_path, sim.config[stim_name]['SpikeFile']))
        stim_cfg_file = os.path.splitext(stim_spike_file)[0] + '.json'
        with open(stim_cfg_file, 'r') as f:
            stim_cfg = json.load(f)
        
        num_patterns = max(stim_cfg['props']['stim_train']) + 1 # len(stim_cfg['props']['pattern_grps'])
        stim_train = stim_cfg['props']['stim_train']
        time_windows = stim_cfg['props']['time_windows']

#         ###
#         #Load manually
#         stim_cfg_file = os.path.join(os.path.split(sim_path)[0], 'input_spikes', 'stimulus_stream__start2000__end61501__rate2__seed12.txt')
#         stim_cfg = pd.read_table(stim_cfg_file, sep=' ', names=['onset', 'pattern'], index_col='onset')
#         stim_cfg = stim_cfg[stim_cfg.index >= 1500] # Filter initial transients
#         pattern_list = np.unique(stim_cfg['pattern'])
#         stim_cfg['pid'] = [np.where(pattern_list == p)[0][0] for p in stim_cfg['pattern']] # Map patterns (str) to indices
#         num_patterns = len(pattern_list)
#         stim_train = stim_cfg['pid'].tolist()
#         time_windows = stim_cfg.index.tolist()
#         ###

        fig, ax = plt.subplots(1, num_patterns, figsize=(3.5 * num_patterns, 3.5), dpi=300)
        if not hasattr(ax, '__iter__'):
            ax = [ax]
        ax[0].set_ylabel(f'Firing rates (Hz)')
        mean_PSTHs = np.zeros((len(syn_classes), num_patterns, num_bins))
        for lidx, lay in enumerate(layers):
            for sclidx, sclass in enumerate(syn_classes):
                gids_sel = np.intersect1d(sim.target_gids, c.cells.ids({Cell.LAYER: lay, Cell.SYNAPSE_CLASS: sclass}))
                stim_spikes = sim.spikes.get(gids=gids_sel)
                num_cells = len(gids_sel)
                
                pattern_spikes = [[]] * num_patterns
                for sidx, pidx in enumerate(stim_train):
                    spikes = stim_spikes[np.logical_and(stim_spikes.index - time_windows[sidx] >= psth_interval[0],
                                                        stim_spikes.index - time_windows[sidx] < psth_interval[-1])]
                    spikes.index = spikes.index - time_windows[sidx] # Re-align to stimulus onset
                    pattern_spikes[pidx] = pattern_spikes[pidx] + [spikes]
                
                num_stim_per_pattern = [len(p) for p in pattern_spikes]
                pattern_spikes = [pd.concat(p) for p in pattern_spikes]
                
                for pidx in range(num_patterns):
                    if num_cells > 0:
                        pattern_PSTH = 1e3 * np.histogram(pattern_spikes[pidx].index, bins=bins)[0] / (psth_bin_size * num_cells * num_stim_per_pattern[pidx])
                        mean_PSTHs[sclidx, pidx, :] = mean_PSTHs[sclidx, pidx, :] + pattern_PSTH / len(layers)
                        ax[pidx].plot(bins[:-1] + 0.5 * psth_bin_size, pattern_PSTH, color=layer_colors[lidx], linestyle=syn_ls[sclidx], label=f'L{lay}-{sclass}' if pidx == num_patterns - 1 else None)
                    ax[pidx].set_xlim((min(bins), max(bins)))
                    ax[pidx].set_xlabel('Time (ms)')
                    ax[pidx].set_title(f'Pattern {pidx} (N={num_stim_per_pattern[pidx]})', fontweight='bold')
        for sclidx, sclass in enumerate(syn_classes):
            for pidx in range(num_patterns):
                ax[pidx].plot(bins[:-1] + 0.5 * psth_bin_size, mean_PSTHs[sclidx, pidx, :], color='k', linestyle=syn_ls[sclidx], linewidth=2, label=f'Mean-{sclass}' if pidx == num_patterns - 1 else None)
        max_lim = max([max(ax[pidx].get_ylim()) for pidx in range(num_patterns)])
        for pidx in range(num_patterns):
            ax[pidx].set_ylim((0, max_lim))
            ax[pidx].plot(np.zeros(2), ax[pidx].get_ylim(), '-', color='darkgrey', linewidth=2, zorder=0)
        label = '  '.join([f'{i[0]}={i[1]}' for i in zip(idx_names, data_idx)])
        fig.suptitle(f'Population PSTHs ({label})', fontweight='bold')
        fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.93), ncol=(len(layers) + 1) * len(syn_classes), fontsize=6)
        fig.tight_layout()
        
        if save_figs:
            save_path = os.path.join(save_path_base, config.name, 'analyses')
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            output_file = os.path.join(save_path, sim_campaign_cfg.name + f'__PSTHs_{idx}.png')
            plt.savefig(output_file)
    except FileNotFoundError:
        print('Results or stimulus config file not found ... SKIPPING')

In [ ]:
# Population PSTHs (per EXC/INH & single repetitions)

c = Circuit(config.circuit_config)
syn_classes = np.unique(c.cells.get(properties=Cell.SYNAPSE_CLASS))
syn_ls = ['-', '--'] # Linestyles
syn_colors = ['r', 'b']

t_len = np.diff(psth_interval)[0]
num_bins = np.round(t_len / psth_bin_size).astype(int)
bins = np.arange(num_bins + 1) * psth_bin_size + psth_interval[0]

for idx, (data_idx, data_path) in enumerate(config.to_series().iteritems()):
    if not isinstance(data_idx, tuple):
        data_idx = (data_idx,)  # if index is atomic (just a number) make tuple out of it
    try:
        sim_path = os.path.join(config.path_prefix, data_path)
        blue_config_file = os.path.join(sim_path, 'BlueConfig')
        
        sim = Simulation(blue_config_file)        
        stim_spike_file = os.path.normpath(os.path.join(sim_path, sim.config[stim_name]['SpikeFile']))
        stim_cfg_file = os.path.splitext(stim_spike_file)[0] + '.json'
        with open(stim_cfg_file, 'r') as f:
            stim_cfg = json.load(f)
        
        num_patterns = num_patterns = max(stim_cfg['props']['stim_train']) + 1 # len(stim_cfg['props']['pattern_grps'])
        stim_train = stim_cfg['props']['stim_train']
        time_windows = stim_cfg['props']['time_windows']

#         ###
#         #Load manually
#         stim_cfg_file = os.path.join(os.path.split(sim_path)[0], 'input_spikes', 'stimulus_stream__start2000__end61501__rate2__seed12.txt')
#         stim_cfg = pd.read_table(stim_cfg_file, sep=' ', names=['onset', 'pattern'], index_col='onset')
#         stim_cfg = stim_cfg[stim_cfg.index >= 1500] # Filter initial transients
#         pattern_list = np.unique(stim_cfg['pattern'])
#         stim_cfg['pid'] = [np.where(pattern_list == p)[0][0] for p in stim_cfg['pattern']] # Map patterns (str) to indices
#         num_patterns = len(pattern_list)
#         stim_train = stim_cfg['pid'].tolist()
#         time_windows = stim_cfg.index.tolist()
#         ###

        fig, ax = plt.subplots(1, num_patterns, figsize=(3.5 * num_patterns, 3.5), dpi=300)
        if not hasattr(ax, '__iter__'):
            ax = [ax]
        ax[0].set_ylabel(f'Firing rates (Hz)')
        mean_PSTHs = np.zeros((len(syn_classes), num_patterns, num_bins))
        for sclidx, sclass in enumerate(syn_classes):
            gids_sel = np.intersect1d(sim.target_gids, c.cells.ids({Cell.SYNAPSE_CLASS: sclass}))
            stim_spikes = sim.spikes.get(gids=gids_sel)
            num_cells = len(gids_sel)

            pattern_spikes = [[]] * num_patterns
            for sidx, pidx in enumerate(stim_train):
                spikes = stim_spikes[np.logical_and(stim_spikes.index - time_windows[sidx] >= psth_interval[0],
                                                    stim_spikes.index - time_windows[sidx] < psth_interval[-1])]
                spikes.index = spikes.index - time_windows[sidx] # Re-align to stimulus onset
                pattern_spikes[pidx] = pattern_spikes[pidx] + [spikes]

            num_stim_per_pattern = [len(p) for p in pattern_spikes]

            for pidx in range(num_patterns):
                if num_cells > 0:
                    pattern_PSTHs = [1e3 * np.histogram(spk.index, bins=bins)[0] / (psth_bin_size * num_cells) for spk in pattern_spikes[pidx]]
                    mean_PSTHs[sclidx, pidx, :] = np.mean(pattern_PSTHs, 0)
                    for pattern_PSTH in pattern_PSTHs:
                        ax[pidx].plot(bins[:-1] + 0.5 * psth_bin_size, pattern_PSTH, color=syn_colors[sclidx], linestyle=syn_ls[sclidx], linewidth=1, alpha=0.25)
                ax[pidx].set_xlim((min(bins), max(bins)))
                ax[pidx].set_xlabel('Time (ms)')
                ax[pidx].set_title(f'Pattern {pidx} (N={num_stim_per_pattern[pidx]})', fontweight='bold')
        for sclidx, sclass in enumerate(syn_classes):
            for pidx in range(num_patterns):
                ax[pidx].plot(bins[:-1] + 0.5 * psth_bin_size, mean_PSTHs[sclidx, pidx, :], color=syn_colors[sclidx], linestyle=syn_ls[sclidx], linewidth=2, label=f'{sclass}' if pidx == num_patterns - 1 else None)
        max_lim = max([max(ax[pidx].get_ylim()) for pidx in range(num_patterns)])
        for pidx in range(num_patterns):
            ax[pidx].set_ylim((0, max_lim))
            ax[pidx].plot(np.zeros(2), ax[pidx].get_ylim(), '-', color='darkgrey', linewidth=2, zorder=0)
        label = '  '.join([f'{i[0]}={i[1]}' for i in zip(idx_names, data_idx)])
        fig.suptitle(f'Population PSTHs ({label})', fontweight='bold')
        fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.93), ncol=(len(layers) + 1) * len(syn_classes), fontsize=6)
        fig.tight_layout()
        
        if save_figs:
            save_path = os.path.join(save_path_base, config.name, 'analyses')
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            output_file = os.path.join(save_path, sim_campaign_cfg.name + f'__Responses_{idx}.png')
            plt.savefig(output_file)
    except FileNotFoundError:
        print('Results or stimulus config file not found ... SKIPPING')

In [ ]:
# Average response rates over time (average rates + single repetitions) + contrast tuning curves (if available)
c = Circuit(config.circuit_config)
syn_classes = np.unique(c.cells.get(properties=Cell.SYNAPSE_CLASS))
syn_colors = ['r', 'b']

for idx, (data_idx, data_path) in enumerate(config.to_series().iteritems()):
    if not isinstance(data_idx, tuple):
        data_idx = (data_idx,)  # if index is atomic (just a number) make tuple out of it
    try:
        sim_path = os.path.join(config.path_prefix, data_path)
        blue_config_file = os.path.join(sim_path, 'BlueConfig')
        
        sim = Simulation(blue_config_file)        
        stim_spike_file = os.path.normpath(os.path.join(sim_path, sim.config[stim_name]['SpikeFile']))
        stim_cfg_file = os.path.splitext(stim_spike_file)[0] + '.json'
        with open(stim_cfg_file, 'r') as f:
            stim_cfg = json.load(f)

        num_patterns = max(stim_cfg['props']['stim_train']) + 1 # len(stim_cfg['props']['pattern_grps'])
        stim_train = stim_cfg['props']['stim_train']
        time_windows = stim_cfg['props']['time_windows']
        response_rates = [[]] * len(syn_classes)
        response_rates_split0 = [[]] * len(syn_classes) # Response interval split into two halves
        response_rates_split1 = [[]] * len(syn_classes) # Response interval split into two halves

        fig, ax = plt.subplots(1, num_patterns, figsize=(3.5 * num_patterns, 3.5), dpi=300)
        if not hasattr(ax, '__iter__'):
            ax = [ax]
        ax[0].set_ylabel(f'Firing rates (Hz)')
        for sclidx, sclass in enumerate(syn_classes):
            gids_sel = np.intersect1d(sim.target_gids, c.cells.ids({Cell.SYNAPSE_CLASS: sclass}))
            stim_spikes = sim.spikes.get(gids=gids_sel)
            num_cells = len(gids_sel)

            pattern_rates = [[]] * num_patterns
            pattern_rates_split0 = [[]] * num_patterns
            pattern_rates_split1 = [[]] * num_patterns
            for sidx, pidx in enumerate(stim_train):
                spikes = stim_spikes[np.logical_and(stim_spikes.index - time_windows[sidx] >= response_interval[0],
                                                    stim_spikes.index - time_windows[sidx] < response_interval[-1])]
                pattern_rates[pidx] = pattern_rates[pidx] + [1000.0 * len(spikes) / (num_cells * (response_interval[-1] - response_interval[0]))]

                spikes = stim_spikes[np.logical_and(stim_spikes.index - time_windows[sidx] >= response_interval[0],
                                                    stim_spikes.index - time_windows[sidx] < response_interval[0] + 0.5 * (response_interval[-1] - response_interval[0]))]
                pattern_rates_split0[pidx] = pattern_rates_split0[pidx] + [1000.0 * len(spikes) / (num_cells * 0.5 * (response_interval[-1] - response_interval[0]))]

                spikes = stim_spikes[np.logical_and(stim_spikes.index - time_windows[sidx] >= response_interval[0] + 0.5 * (response_interval[-1] - response_interval[0]),
                                                    stim_spikes.index - time_windows[sidx] < response_interval[-1])]
                pattern_rates_split1[pidx] = pattern_rates_split1[pidx] + [1000.0 * len(spikes) / (num_cells * 0.5 * (response_interval[-1] - response_interval[0]))]

            response_rates[sclidx] = response_rates[sclidx] + pattern_rates
            response_rates_split0[sclidx] = response_rates_split0[sclidx] + pattern_rates_split0
            response_rates_split1[sclidx] = response_rates_split1[sclidx] + pattern_rates_split1

        for pidx in range(num_patterns):
            for sclidx in range(len(syn_classes)):
                ax[pidx].bar(sclidx, np.mean(response_rates[sclidx][pidx]), color=syn_colors[sclidx])
                ax[pidx].plot(np.full(len(response_rates[sclidx][pidx]), sclidx), response_rates[sclidx][pidx], 'ko')
                ax[pidx].plot([np.full(len(response_rates[k][pidx]), k) for k in range(len(syn_classes))], [response_rates[k][pidx] for k in range(len(syn_classes))], 'k-')
                ax[pidx].set_xticks(range(len(syn_classes)))
            ax[pidx].set_xticklabels(syn_classes)
            ax[pidx].set_title(f'Pattern {pidx} (N={len(response_rates[0][pidx])})', fontweight='bold')
        max_lim = max([max(ax[pidx].get_ylim()) for pidx in range(num_patterns)])
        for pidx in range(num_patterns):
            ax[pidx].set_ylim((0, max_lim))
        label = '  '.join([f'{i[0]}={i[1]}' for i in zip(idx_names, data_idx)])
        fig.suptitle(f'Response rates ({label})', fontweight='bold')
        fig.tight_layout()

        if save_figs:
            save_path = os.path.join(save_path_base, config.name, 'analyses')
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            output_file = os.path.join(save_path, sim_campaign_cfg.name + f'__ResponseRates_{idx}.png')
            plt.savefig(output_file)

        # Plot average contrast tuning curve
        if 'contrast_levels' in stim_cfg['cfg']:
            R = lambda c, m, n, Rmax, c50: (Rmax * c**n) / (c**n + c50**n) + m # Sigmoidal response function R(c) [Shapiro et al. 2021]
            c_data = stim_cfg['cfg']['contrast_levels']
            c_fit = np.arange(min(c_data), max(c_data), 0.01)

            res_path = os.path.join(save_path, 'output', 'contrast_tuning')
            figs_path = os.path.join(res_path, 'figs')
            if not os.path.exists(res_path):
                os.makedirs(res_path)
            if not os.path.exists(figs_path):
                os.makedirs(figs_path)
            sim_spec = '__'.join([f'{k}_{v}' for k, v in zip(idx_names, data_idx)])
            sim_id = os.path.split(sim_path)[-1]

            plt.figure(figsize=(10, 3))
            for sclidx, sclass in enumerate(syn_classes):
                contrast_tuning_dict = {'c_data': c_data, 'r_data': [], 'p_opt': []}

                plt.subplot(1, len(syn_classes), sclidx + 1)
                r_data = np.mean(response_rates_split0[sclidx], 1) # First half
                p_opt, _ = curve_fit(R, c_data, r_data, p0=[0.0, 1.0, 1.0, 0.25], bounds=(0.0, [np.inf, np.inf, np.inf, 1.0]), maxfev=10000)
                plt.plot(c_fit, R(c_fit, *p_opt), '--', color='tab:blue', label='T$_{resp}$=' + f'{response_interval[0]}-{np.round(response_interval[0] + 0.5 * (response_interval[-1] - response_interval[0])).astype(int)}ms')
                plt.plot(c_data, r_data, '.', color='tab:blue')
                contrast_tuning_dict['r_data'].append(r_data)
                contrast_tuning_dict['p_opt'].append({k: v for k, v in zip(R.__code__.co_varnames[1:], p_opt)})

                r_data = np.mean(response_rates_split1[sclidx], 1) # Second half
                p_opt, _ = curve_fit(R, c_data, r_data, p0=[0.0, 1.0, 1.0, 0.25], bounds=(0.0, [np.inf, np.inf, np.inf, 1.0]), maxfev=10000)
                plt.plot(c_fit, R(c_fit, *p_opt), '--', color='tab:orange', label='T$_{resp}$=' + f'{np.round(response_interval[0] + 0.5 * (response_interval[-1] - response_interval[0])).astype(int)}-{response_interval[-1]}ms')
                plt.plot(c_data, r_data, '.', color='tab:orange')
                contrast_tuning_dict['r_data'].append(r_data)
                contrast_tuning_dict['p_opt'].append({k: v for k, v in zip(R.__code__.co_varnames[1:], p_opt)})

                r_data = np.mean(response_rates[sclidx], 1) # Full response interval
                p_opt, _ = curve_fit(R, c_data, r_data, p0=[0.0, 1.0, 1.0, 0.25], bounds=(0.0, [np.inf, np.inf, np.inf, 1.0]), maxfev=10000)
                plt.plot(c_fit, R(c_fit, *p_opt), color='k', label='T$_{resp}$=' + f'{response_interval[0]}-{response_interval[-1]}ms')
                plt.plot(c_data, r_data, '.', color='k')
                contrast_tuning_dict['r_data'].append(r_data)
                contrast_tuning_dict['p_opt'].append({k: v for k, v in zip(R.__code__.co_varnames[1:], p_opt)})

                plt.gca().set_xscale('log')
                plt.xticks(c_data, labels=c_data)
                plt.gca().minorticks_off()
                plt.ylim([0, None])
                plt.xlabel('Contrast')
                plt.ylabel('Average response rates (Hz)')
                plt.title(f'{sclass} tuning', fontweight='bold')
                plt.legend(fontsize=8)

                # Save results
                res_fn = f'contrast_tuning__{sim.target}_{sclass}__SIM{sim_id}__{sim_spec}.pickle'
                with open(os.path.join(res_path, res_fn), 'wb') as f:
                    pickle.dump(contrast_tuning_dict, f)

            plt.suptitle(f'Contrast tuning ({label})', fontweight='bold')
            plt.tight_layout()
            fig_fn = f'contrast_tuning__{sim.target}__SIM{sim_id}__{sim_spec}.png'
            plt.savefig(os.path.join(figs_path, fig_fn))

    except FileNotFoundError:
        print('Results or stimulus config file not found ... SKIPPING')